In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# 다운 받은 csv를 pandas의 DataFrame 형식으로 불러온다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.drop('id',axis = 1)
test = test.drop('id',axis = 1)

In [4]:
train.fillna(train.mean(), inplace = True)
test.fillna(train.mean(), inplace = True)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print(train.shape)
print(test.shape)

(114, 16)
(228, 15)


In [6]:
discrete_names = ['Species', 'Island', 'Clutch Completion', 'Sex']
continuous_names = ['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)']

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_scaler = scaler.fit_transform(train[continuous_names])
train[continuous_names] = pd.DataFrame(data=train_scaler, columns=continuous_names)

test_scaler = scaler.transform(test[continuous_names])
test[continuous_names] = pd.DataFrame(data=test_scaler, columns=continuous_names)

train[continuous_names].head()

,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo)
0,1.016685,-0.887255,1.161653,-0.775548,0.630951
1,0.922318,1.027037,-0.209242,1.601553,1.629486
2,0.091884,-1.352893,0.476205,-0.391149,-1.533908
3,-0.021357,-1.197680,0.750384,-0.962206,-0.523568
4,0.941191,-0.525091,1.504376,-0.635514,-1.325731


In [8]:
train_label = train['Body Mass (g)']
train.drop(['Body Mass (g)'], axis = 1, inplace = True)

In [9]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

alphas = [0,0.1,1,10,100]

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    
    neg_mse_scores = cross_val_score(ridge, train, train_label, scoring = 'neg_mean_squared_error', cv = 5)
    avg_rmse = np.mean(np.sqrt(-neg_mse_scores))
    print('alpha 값 ', alpha, '일때 평균 rmse :', np.round(avg_rmse,4))

alpha 값  0 일때 평균 rmse : 339.5967
alpha 값  0.1 일때 평균 rmse : 337.4481
alpha 값  1 일때 평균 rmse : 330.4727
alpha 값  10 일때 평균 rmse : 329.2807
alpha 값  100 일때 평균 rmse : 405.4389


In [10]:
ridge = Ridge(alpha = 10)
ridge.fit(train, train_label)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['Body Mass (g)'] = ridge.predict(test)
sample_submission.to_csv('Penguin_final_1.csv',index=False)